In [ ]:
import varseek as vk

In [ ]:
# Required
mutations = "/home/jrich/data/varseek_data_fresh/reference/cosmic/CancerMutationCensus_AllData_Tsv_v100_GRCh37/CancerMutationCensus_AllData_v100_GRCh37_mutation_workflow_with_cdna.csv"  # "cosmic_cmc"  # file path to mutations csv/tsv file OR one of the supported databases - eg cosmic_cmc
sequences = "/home/jrich/data/varseek_data_fresh/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.cdna.all.fa"  # cdna  # file path to reference sequence fasta file OR, only if 'mutations' is in the supported databases, one of the following options is also supported: "cds", "cdna", "genome", "cdna_and_genome" - sequences for gget mutate
rnaseq_fastq_files = ""  # pass as a single string or as a list of strings (if paired-end, then pass each pair sequentially with R1 preceding R2)


# Optional
verbose=True
w=54  # window size for varseek build (should be at least 1 less than kallisto k)
k=59
threads = 32
remove_Ns = True
strand = "unstranded"  # "forward", "reverse", or "unstranded"
strandedness = False  # strandedness for gget mutate and the building of the kb index (True = strandedness matters i.e., treat f and rc as 2 different sequences; False = strandedness does not matter i.e., treat f and rc as the same 

# vk build
cosmic_version = 100  # COSMIC version for gget cosmic
# os.environ['COSMIC_EMAIL'] = 'your_email'  # to avoid being prompted for email in varseek build
# os.environ['COSMIC_PASSWORD'] = 'your_password'  # to avoid being prompted for password in varseek build

# vk info
columns_to_include="all"
dlist_reference_source = "ensembl_grch37_release93"  # ensembl_grchNUMBER_releaseNUMBER or t2t - eg ensembl_grch37_release93
near_splice_junction_threshold=10
save_exploded_df=True

# vk filter
fasta_filters = [
    "dlist_substring-equal=none",  # filter out mutations which are a substring of the reference genome
    "pseudoaligned_to_human_reference_despite_not_truly_aligning-isnottrue",  # filter out mutations which pseudoaligned to human genome despite not truly aligning
    "dlist-equal=none",  #*** erase eventually when I want to d-list  # filter out mutations which are capable of being d-listed (given that I filter out the substrings above)
    "number_of_kmers_with_overlap_to_other_mcrs_items_in_mcrs_reference-max=999999",  # filter out mutations which overlap with other MCRSs in the reference
    "number_of_mcrs_items_with_overlapping_kmers_in_mcrs_reference-max=999999",  # filter out mutations which overlap with other MCRSs in the reference
    "longest_homopolymer_length-max=999999",  # filters out MCRSs with repeating single nucleotide - eg 6
    "triplet_complexity-min=0"  # filters out MCRSs with repeating triplets - eg 0.2
]

# kb ref
dlist_kb_argument = "None"  # path to dlist fasta file or "None" (including the quotes)

# vk fastqpp
trim_edges_off_reads = False
minimum_base_quality_trim_reads=13
qualified_quality_phred = None
unqualified_percent_limit = None
n_base_limit = None
length_required = None
replace_low_quality_bases_with_N = False
minimum_base_quality_replace_with_N=13
split_reads_by_Ns = False
run_fastqc = False

# kb count
assay = "bulk"  # "bulk" or "sc"  #!!! implement
parity = "single"  # single or paired

# vk clean
minimum_count_filter = None  #!!! still need to tune this  
use_binary_matrix = False
drop_zero_columns = False
filter_cells_by_min_counts = True  # True for auto-filter, numeric for set filter, None/False otherwise
filter_cells_by_min_genes = 200
filter_genes_by_min_cells = 3
filter_cells_by_max_mt_content = False
doublet_detection = True
remove_doublets = False
do_cpm_normalization = True
split_reads_by_Ns = False
dlist_file = None
mutation_metadata_df_columns = None  #!!! replace


# Paths
out_dir_base = "/home/jrich/data/varseek_data_fresh"
run_name = "vk_build_pipeline_grch37_nov12"

# vk info
bowtie_path="/home/jrich/opt/bowtie2-2.5.4/bowtie2-2.5.4-linux-x86_64"
mutations_csv="/home/jrich/data/varseek_data_fresh/reference/cosmic/CancerMutationCensus_AllData_Tsv_v100_GRCh37/CancerMutationCensus_AllData_v100_GRCh37_with_cdna.csv"
reference_cdna_fasta="/home/jrich/data/varseek_data_fresh/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.cdna.all.fa"  # the one that matches up to mutation df annotations
reference_genome_fasta="/home/jrich/data/varseek_data_fresh/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa"  # the one that matches up to mutation df annotations
gtf_path="/home/jrich/data/varseek_data_fresh/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.87.gtf"  # the one that matches up to mutation df annotations
seqtk="seqtk"
fastp="fastp"

# vk clean
mutation_metadata_df_path = None  #!!! replace
standard_index = ""
standard_t2g = ""

In [ ]:
vk_ref_output_dict = vk.ref()

index = vk_ref_output_dict["index"]
t2g = vk_ref_output_dict["t2g"]

In [ ]:
vk_count_output_dict = vk.clean()
print(vk_count_output_dict)